# Preparation

## import tools

In [1]:
import os, numpy as np
import histogram.hdf as hh
import mcvine, mcvine.components

from matplotlib import pyplot as plt
%matplotlib notebook

/SNS/software/miniconda2/envs/py2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## work dir

In [2]:
workdir = os.path.expanduser("~/simulations/mcvine/demo-ms")
!mkdir -p {workdir}
%cd {workdir}

/SNS/users/lj7/simulations/mcvine/demo-ms


# Sample

In [4]:
%%file Al-scatterer.xml
<?xml version="1.0"?>

<!DOCTYPE scatterer>

<!-- weights: absorption, scattering, transmission -->
<homogeneous_scatterer mcweights="0, 1, 0">
  
  <E_Q_Kernel 
     E_Q="0" 
     S_Q="1" 
     Qmin="0./angstrom"
     Qmax="10./angstrom"
     />
  
</homogeneous_scatterer>

Overwriting Al-scatterer.xml


In [5]:
%%file Al.xyz
4
4.04 0 0   0 4.04 0   0 0 4.04
Al 0  0  0
Al 0.5 0.5 0
Al 0.5 0 0.5
Al 0 0.5 0.5

Overwriting Al.xyz


In [6]:
%%file sampleassembly.xml
<?xml version="1.0"?>

<!DOCTYPE SampleAssembly>

<SampleAssembly name="Al"
   max_multiplescattering_loops_among_scatterers="1"
   max_multiplescattering_loops_interactM_path1="2"
   min_neutron_probability=".005"
>
  
  <PowderSample name="Al" type="sample">
    <Shape>
      <hollowCylinder in_radius="5*cm" out_radius="5.1*cm" height="10*cm" />
    </Shape>
    <Phase type="crystal">
      <ChemicalFormula>Al</ChemicalFormula>
      <xyzfile>Al.xyz</xyzfile>
    </Phase>
  </PowderSample>
  
  <LocalGeometer registry-coordinate-system="InstrumentScientist">
    <Register name="Al" position="(0,0,0)" orientation="(0,0,0)"/>
  </LocalGeometer>
<!--
  <environment>
    <temperature>300*K</temperature>
    <pressure>1.*atm</pressure>
  </environment>
-->

</SampleAssembly>

Overwriting sampleassembly.xml


# Instrument

## create script

In [7]:
%%file myinstrument.py
import mcvine, mcvine.components
instrument = mcvine.instrument()
source = mcvine.components.sources.Source_simple('source', E0=50., dE=0.5)
instrument.append(source, position=(0,0,0))
sample = mcvine.components.samples.SampleAssemblyFromXml('sample', 'sampleassembly.xml')
instrument.append(sample, position=(0,0,1))
ns = mcvine.components.monitors.NeutronToStorage('ns', 'scattered.mcvine')
instrument.append(ns, position=(0,0,1))

Overwriting myinstrument.py


## check/debug script

In [8]:
from mcvine import run_script

In [14]:
rm -rf debug

In [15]:
run_script.run1('./myinstrument.py', 'debug', ncount=10, overwrite_datafiles=False, multiple_scattering=True)

/SNS/software/miniconda2/envs/py2/lib/python2.7/site-packages/mcvine/run_script.py:125: UserWarning: Developer: <mccomposite.mccompositebp.CompositeNeutronScatterer object at 0x7f8f149089f0> does not implement method "create_pps"
  warnings.warn('Developer: %s does not implement method "create_pps"' % comp)


In [16]:
!mcvine neutronstorage count ./debug/step0/scattered.mcvine

50


## production run

In [18]:
%%time
!rm -rf work
run_script.run_mpi('./myinstrument.py', 'work', ncount=1e3, nodes=10, overwrite_datafiles=False, multiple_scattering=True)

CPU times: user 4.95 ms, sys: 12.9 ms, total: 17.8 ms
Wall time: 7.51 s


In [20]:
!mcvine neutronstorage count work/scattered.mcvine

4998
